In [1]:
import numpy as np
import pandas as pd
import keras
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
df= pd.read_csv("/home/samridhi/boatgang/dataset/data.csv", header=None, skiprows=1)
df.head()

,0,1
0,TCCGATATGTACCCGGACGTAATCCGCAGAATAAAGTATCTAAAAG...,1
1,GGTAACGAAGTGGTTACATTATGCTCTTCCGTTTCGGATTGTGAAC...,1
2,GCTTCGTCACGGACGTCCCTCGATTAGACTCGATATGTACCTAACG...,1
3,ATGCTTCTACACGCATTATACAGCCCCGCGTTTTAGTCCTTTTCAG...,1
4,GGACTAGTTGAGGAACAATGAACTCTTTTCAATTGAAACCGAGTTA...,1


In [3]:
df.shape

(60000, 2)

In [4]:
temp_sequences=df[0]
temp_target=df[1]

In [5]:
dict_i = {"A" : 0, "C": 1, "G": 2, "T" : 3}

In [6]:
def one_hot_encode(s):
    i=0
    ohe = np.zeros((len(s), 4))
    for k in s:
        ohe[i,dict_i[k]] = 1
        i+=1
    return ohe

In [7]:
ohe_sequences = np.array([one_hot_encode(s) for s in temp_sequences])
print ohe_sequences
ohe_sequences.shape

[[[0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 ...

 [[0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]]


(60000, 200, 4)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout,Flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

We used the Keras deep learning framework (https://github.com/keras-team/keras) to train a CNN to classify
the sequences. The sequences were represented using a one-hot encoding with 4 channels (A, C, G and T). The
CNN architecture is as follows: Layer 1 is a convolutional layer with 40 filters of size 19 and ReLU activation
operating on one-hot encoded input sequences. Layer 2 is a max pooling layer of pool length 10. Layer 3 is a
fully connected layer of size 200 with dropout (p=0.5) and ReLU activation. Layer 4 is a fully connected layer
with a sigmoid activation. The model was trained with the Adam optimizer and binary cross-entropy loss until
no improvement was seen for 3 epochs on the validation set. The datasets, code and model are available at

In [18]:
model = Sequential()
model.add(Conv1D(filters=40, kernel_size=10, activation='relu', input_shape=(200,4)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 191, 40)           1640      
Total params: 1,640
Trainable params: 1,640
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.add(MaxPooling1D(pool_size=191, strides=1))

In [20]:
model.add(Dense(32,  activation='relu'))
model.add(Flatten())

In [21]:
model.add(Dense(1, activation = "sigmoid"))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 191, 40)           1640      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 40)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1, 32)             1312      
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,985
Trainable params: 2,985
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [24]:
ohe_sequences=shuffle(ohe_sequences)

In [25]:
data_train, data_test, labels_train, labels_test = train_test_split(ohe_sequences, temp_target, test_size=0.30, random_state=42)

In [26]:
model.fit(data_train, labels_train, epochs=10, verbose=1)

Epoch 1/10
42000/42000 [==============================] - 15s 365us/step - loss: 0.6949 - acc: 0.4996
Epoch 2/10
42000/42000 [==============================] - 15s 356us/step - loss: 0.6936 - acc: 0.5035
Epoch 3/10
42000/42000 [==============================] - 16s 385us/step - loss: 0.6933 - acc: 0.5072
Epoch 4/10
42000/42000 [==============================] - 16s 384us/step - loss: 0.6930 - acc: 0.5089
Epoch 5/10
42000/42000 [==============================] - 16s 390us/step - loss: 0.6923 - acc: 0.5195
Epoch 6/10
42000/42000 [==============================] - 17s 416us/step - loss: 0.6914 - acc: 0.5235
Epoch 7/10
42000/42000 [==============================] - 17s 408us/step - loss: 0.6905 - acc: 0.5281
Epoch 8/10
42000/42000 [==============================] - 17s 401us/step - loss: 0.6892 - acc: 0.5382
Epoch 9/10
42000/42000 [==============================] - 18s 417us/step - loss: 0.6877 - acc: 0.5440
Epoch 10/10
42000/42000 [==============================] - 17s 395us/step - loss: 

In [27]:
scores = model.evaluate(data_test, labels_test)
print ("Test loss ", scores[0])
print ("Test acc ", scores[1])

18000/18000 [==============================] - 1s 42us/step
('Test loss ', 0.6993947748608059)
('Test acc ', 0.49777777777777776)


In [ ]:
model.get_config()